## Language Models: Auto Complete

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/NLP/Language Modelling/Probabilistic Language modelling/en_US.twitter.txt") as f:
  data = f.read()

In [ ]:
print("Data type:", type(data))
print("Number of letters:", len(data))
print(data[:300])
print("============")
print(data[-300:])

Data type: <class 'str'>
Number of letters: 3335477
How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.
When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.
they've decided its more fun if I don't.
So Tired D; Played Lazer Tag & Ran A 
ust had one a few weeks back....hopefully we will be back soon! wish you the best yo
Colombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”
#GutsiestMovesYouCanMake Giving a cat a bath.
Coffee after 5 was a TERRIBLE idea.



In [ ]:
def split_to_sentences(data):
  sentences = data.split('\n')
  # remove the leading and trailing spaces from each sentences
  sentences = [s.strip() for s in sentences]
  # drop sentences if they are empty strings
  sentences = [s for s in sentences if len(s) > 0]
  return sentences

In [ ]:
x = data[0:300]
x

"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

In [ ]:
print(x)

How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.
When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.
they've decided its more fun if I don't.
So Tired D; Played Lazer Tag & Ran A 


In [ ]:
split_to_sentences(x)

['How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.',
 "When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.",
 "they've decided its more fun if I don't.",
 'So Tired D; Played Lazer Tag & Ran A']

In [ ]:
def tokenize_sentences(sentences):
  tokenized_sentences = []

  for sentence in sentences:
    # Convert to lowercase letters
    sentence = sentence.lower()
   
    # Convert into a list of words
    tokenized = nltk.word_tokenize(sentence)

    # append the list of words to the list of lists
    tokenized_sentences.append(tokenized)

  return tokenized_sentences


In [ ]:
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

In [ ]:
def get_tokenized_data(data):
  # get the sentences by splitting up the data
  sentences = split_to_sentences(data)

  # get the list of lists of tokens by tokenizing the sentences
  tokenized_sentences = tokenize_sentences(sentences)

  return tokenized_sentences


In [ ]:
x = "Sky is blue.\nLeaves are green\nRoses are red."
get_tokenized_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

## Split into train and test data

In [ ]:
tokenized_data = get_tokenized_data(data)
random.seed(87)
random.shuffle(tokenized_data)

train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [ ]:
print("{} data are split into {} train and {} test set".format(
    len(tokenized_data), len(train_data), len(test_data)))

print("First training sample:")
print(train_data[0])
      
print("First test sample")
print(test_data[0])

47961 data are split into 38368 train and 9593 test set
First training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']
First test sample
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']


In [ ]:
def count_words(tokenized_sentences):
  """
  Count the number of word appearance in the tokenized sentences
  """

  word_counts = {}

  for sentence in tokenized_sentences:
    for token in sentence:
      if token not in word_counts.keys():
        word_counts[token] = 1
      else:
        word_counts[token] += 1
  
  return word_counts

In [ ]:
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
count_words(tokenized_sentences)

{'sky': 1,
 'is': 1,
 'blue': 1,
 '.': 3,
 'leaves': 1,
 'are': 2,
 'green': 1,
 'roses': 1,
 'red': 1}

## Handlign Out of Vocabulary words

In [ ]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
  closed_vocab = []

  word_counts = count_words(tokenized_sentences)

  for word, cnt in word_counts.items():
    if cnt >= count_threshold:
      closed_vocab.append(word)
  
  return closed_vocab

In [ ]:
tokenized_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]
tmp_closed_vocab = get_words_with_nplus_frequency(tokenized_sentences, count_threshold=2)
print(f"Closed vocabulary:")
print(tmp_closed_vocab)

Closed vocabulary:
['.', 'are']


In [ ]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token = "<unk>"):
  vocabulary = set(vocabulary)

  replaced_tokenized_sentences = []

  for sentence in tokenized_sentences:
    replaced_sentence = []
  
    for token in sentence:
      if token in vocabulary:
        replaced_sentence.append(token)
      else:
        replaced_sentence.append(unknown_token)
    
    replaced_tokenized_sentences.append(replaced_sentence)

  return replaced_tokenized_sentences

In [ ]:
tokenized_sentences = [["dogs", "run"], ["cats", "sleep"]]
vocabulary = ["dogs", "sleep"]
tmp_replaced_tokenized_sentences = replace_oov_words_by_unk(tokenized_sentences, vocabulary)

In [ ]:
print(f"Original sentence:")
print(tokenized_sentences)
print(f"tokenized_sentences with less frequent words converted to '<unk>':")
print(tmp_replaced_tokenized_sentences)

Original sentence:
[['dogs', 'run'], ['cats', 'sleep']]
tokenized_sentences with less frequent words converted to '<unk>':
[['dogs', '<unk>'], ['<unk>', 'sleep']]


In [ ]:
def preprocess_data(train_data, test_data, count_threshold):
  vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)

  train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)

  test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)

  return train_data_replaced, test_data_replaced, vocabulary

In [ ]:
tmp_train = [['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]
tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_repl, tmp_test_repl, tmp_vocab = preprocess_data(tmp_train, tmp_test, count_threshold = 1)

print("tmp_train_repl")
print(tmp_train_repl)
print()
print("tmp_test_repl")
print(tmp_test_repl)
print()
print("tmp_vocab")
print(tmp_vocab)

tmp_train_repl
[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green']]

tmp_test_repl
[['<unk>', 'are', '<unk>', '.']]

tmp_vocab
['sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


In [ ]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, minimum_freq)

In [ ]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the', 'team', 'local', 'company', 'and', 'quality', 'production']

First preprocessed test sample:
['that', 'picture', 'i', 'just', 'seen', 'whoa', 'dere', '!', '!', '>', '>', '>', '>', '>', '>', '>']

First 10 vocabulary:
['i', 'personally', 'would', 'like', 'as', 'our', 'official', 'glove', 'of', 'the']

Size of vocabulary: 14824


## Develop a n-gram based language model

In [ ]:
def count_n_grams(data, n, start_token = '<s>', end_token = '</s>'):
  """
  Count all n-grams in the data
  """

  n_grams = {}

  for sentence in data:
    # prepend start token n times and append end token one time
    sentence = [start_token] * n + sentence + [end_token]

    # convert list to tuple so that the sequence of words can be used as a key in the dictionary
    sentence = tuple(sentence)

    # Use value of m to denote the number of n grams in the current sentence
    m = len(sentence) if n == 1 else len(sentence) - 1

    for i in range(m):
      n_gram = sentence[i: i + n]

      # check if the n-gram is in the dictionary
      # if present, increase the value else set the value of n-gram to 1
      if n_gram in n_grams.keys():
        n_grams[n_gram] += 1
      else:
        n_grams[n_gram] = 1 

  return n_grams


In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('</s>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '</s>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


In [ ]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k = 1.0):
  """
  Estimate the probabilities of the next word using the n-gram counts with k-smoothing
  """

  # convert the list to tuple to use it as a dictionary key
  previous_n_gram = tuple(previous_n_gram)

  # set the denominator
  # if the previous n-gram exists in the dictionary of n-gram counts, get its coun
  # else set the count to zero
  # use the dictionary that has counts for n-grams
  previous_n_gram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0

  # calculate the denominator useing the count fo the previous n gram and apply k-smoothing
  denominator = previous_n_gram_count + k * vocabulary_size

  # define n plus 1 gram as the previous n-gram plus the current word as a tuple
  n_plus1_gram = previous_n_gram + (word, )

  # set the count to the count in the dictionary
  # 0 if not in the dictionary
  # use the dictionary that has counts for the n-gram plus current word
  n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0

  # define the numerator using the counf of the n-gram plus current word and apply smoothing
  numerator = n_plus1_gram_count + k

  # calculate the probability as the numberator divided by the denominator
  probability = numerator / denominator

  return probability

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
tmp_prob = estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words), k=1)

print(f"The estimated probability of word 'cat' given the previous n-gram 'a' is: {tmp_prob:.4f}")

The estimated probability of word 'cat' given the previous n-gram 'a' is: 0.3333


Here, estimated probability = (2 + 1) / (2 + 1 * 7) = 0.33333


In [ ]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k = 1.0):
  """
  Estimate the probabilities of next words using the n-gram counts with k-smoothing
  """
  # convert the list to tuple to use it as a dictionary key
  previous_n_gram = tuple(previous_n_gram)

  # add </s> and <unk> to the vocabulary
  # <s> is not needed as it should not appear as the next word
  vocabulary = vocabulary + ["</s>", "<unk>"]
  vocabular_size = len(vocabulary)

  probabilities = {}
  for word in vocabulary:
    probability = estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabular_size, k = k)
    probabilities[word] = probability
  
  return probabilities

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'cat': 0.2727272727272727,
 'a': 0.09090909090909091,
 'is': 0.09090909090909091,
 'i': 0.09090909090909091,
 'dog': 0.09090909090909091,
 'like': 0.09090909090909091,
 'this': 0.09090909090909091,
 '</s>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [ ]:
unique_words # this the vocabulary in the above case

['cat', 'a', 'is', 'i', 'dog', 'like', 'this']

In [ ]:
unigram_counts

{('<s>',): 2,
 ('i',): 1,
 ('like',): 2,
 ('a',): 2,
 ('cat',): 2,
 ('</s>',): 2,
 ('this',): 1,
 ('dog',): 1,
 ('is',): 1}

In [ ]:
bigram_counts

{('<s>', '<s>'): 2,
 ('<s>', 'i'): 1,
 ('i', 'like'): 1,
 ('like', 'a'): 2,
 ('a', 'cat'): 2,
 ('cat', '</s>'): 2,
 ('<s>', 'this'): 1,
 ('this', 'dog'): 1,
 ('dog', 'is'): 1,
 ('is', 'like'): 1}

In [ ]:
trigram_counts = count_n_grams(sentences, 3)
estimate_probabilities(["<s>", "<s>"], bigram_counts, trigram_counts, unique_words, k = 1)

{'cat': 0.09090909090909091,
 'a': 0.09090909090909091,
 'is': 0.09090909090909091,
 'i': 0.18181818181818182,
 'dog': 0.09090909090909091,
 'like': 0.09090909090909091,
 'this': 0.18181818181818182,
 '</s>': 0.09090909090909091,
 '<unk>': 0.09090909090909091}

In [ ]:
trigram_counts

{('<s>', '<s>', '<s>'): 2,
 ('<s>', '<s>', 'i'): 1,
 ('<s>', 'i', 'like'): 1,
 ('i', 'like', 'a'): 1,
 ('like', 'a', 'cat'): 2,
 ('a', 'cat', '</s>'): 2,
 ('cat', '</s>'): 2,
 ('<s>', '<s>', 'this'): 1,
 ('<s>', 'this', 'dog'): 1,
 ('this', 'dog', 'is'): 1,
 ('dog', 'is', 'like'): 1,
 ('is', 'like', 'a'): 1}

## Count and probability matrices

In [ ]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
  # add </s> and <UNK> to the vocabulary
  # <s> is omitted since it should not appear as next word
  vocabulary = vocabulary + ["</s>", "<UNK>"]

  # obtain unique n-grams
  n_grams = []
  for n_plus1_gram in n_plus1_gram_counts.keys():
    n_gram = n_plus1_gram[0:-1]
    n_grams.append(n_gram)
  n_grams = list(set(n_grams))

  # mapping from n-gram to row
  row_index = {n_gram : i for i, n_gram in enumerate(n_grams)}

  # mapping from next word to column
  col_index = {word : j for j, word in enumerate(vocabulary)}

  nrow = len(n_grams)
  ncol = len(vocabulary)
  count_matrix = np.zeros((nrow, ncol))
  for n_plus1_gram, count in n_plus1_gram_counts.items():
    n_gram = n_plus1_gram[0:-1]
    word = n_plus1_gram[-1]
    if word not in vocabulary:
      continue
    i = row_index[n_gram]
    j = col_index[word]
    count_matrix[i, j] = count

  count_matrix = pd.DataFrame(count_matrix, index = n_grams, columns = vocabulary)

  return count_matrix


In [ ]:
sentences = [['i', 'like', 'a', 'cat'], ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)

print('bigram counts')
display(make_count_matrix(bigram_counts, unique_words))

bigram counts


,cat,a,is,i,dog,like,this,</s>,<UNK>
"(like,)",0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(this,)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(cat,)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
"(a,)",2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(<s>,)",0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
"(i,)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(is,)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"(dog,)",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
  count_matrix = make_count_matrix(n_plus1_gram_counts, vocabulary)
  # constant is added to each value in count_matrix to make its probability non-zero
  count_matrix += k 
  prob_matrix = count_matrix.div(count_matrix.sum(axis = 1), axis = 0)
  return prob_matrix

In [ ]:
sentences = [['i', 'like', 'a', 'cat'], ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print("bigram probabilities")
display(make_probability_matrix(bigram_counts, unique_words, k=1))

bigram probabilities


,cat,a,is,i,dog,like,this,</s>,<UNK>
"(like,)",0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(this,)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(a,)",0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(<s>,)",0.090909,0.090909,0.090909,0.181818,0.090909,0.090909,0.181818,0.090909,0.090909
"(i,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(is,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(dog,)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [ ]:
print("trigram probabilities")
trigram_counts = count_n_grams(sentences, 3)
display(make_probability_matrix(trigram_counts, unique_words, k=1))

trigram probabilities


,cat,a,is,i,dog,like,this,</s>,<UNK>
"(i, like)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(this, dog)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<s>, <s>)",0.090909,0.090909,0.090909,0.181818,0.090909,0.090909,0.181818,0.090909,0.090909
"(is, like)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(dog, is)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(cat,)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(<s>, this)",0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
"(like, a)",0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(a, cat)",0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909
"(<s>, i)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000


## Perplexity

In [ ]:
def calculate_perplexity(sentence, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k = 1.0):
  """
  calculate the perplexity for a list of sentences
  """
  # length of previous words
  n = len(list(n_gram_counts.keys())[0])

  # prepend <s> and append </s>
  sentence = ["<s>"] * n + sentence + ["</s>"]

  # cast the sentence from a list to a tuple
  sentence = tuple(sentence)

  # length of sentence (after adding)
  N = len(sentence)

  # The variable p will hold hte product that is calculated inside the n-root 
  # Update this in the code below
  product_pi = 1.0

  # index t ranges from n to N-1, inclusive on both ends
  for t in range(n, N):
    # get the n_gram preceding the word at position t
    n_gram = sentence[t - n : t]

    # get the word at position t
    word = sentence[t]

    # estimate the probability of the word given the n-gram using the n-gram counts, n-plus1-gram counts, vocabulary size and smoothing constant
    probability = estimate_probability(word, n_gram, n_gram_counts, n_plus1_gram_counts, len(unique_words), k = 1)

    # Update the product of the probabilities
    # product_pi is the cumulative product of the (1/P) factors that are calculated in the loop
    product_pi *= 1 / probability

  # Take the N-th root of the product
  perplexity = product_pi ** (1/float(N))

  return perplexity 

In [ ]:
sentences = [['i', 'like', 'a', 'cat'], ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)


perplexity_train1 = calculate_perplexity(sentences[0],
                                         unigram_counts, bigram_counts,
                                         len(unique_words), k=1.0)
print(f"Perplexity for first train sample: {perplexity_train1:.4f}")

test_sentence = ['i', 'like', 'a', 'dog']
perplexity_test = calculate_perplexity(test_sentence,
                                       unigram_counts, bigram_counts,
                                       len(unique_words), k=1.0)
print(f"Perplexity for test sample: {perplexity_test:.4f}")

Perplexity for first train sample: 2.8040
Perplexity for test sample: 3.9654


In [ ]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k = 1.0, start_with = None):
  """
  Get suggestion for the next word
  """

  # length of previous words
  n = len(list(n_gram_counts.keys())[0])

  # From the words that the user already typed, get the most recent 'n' words as the previous n-gram
  previous_n_gram = previous_tokens[-n:]

  # Estimate the probabilities that each word in the vocabular is the next word
  probabilities = estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k = k)
  
  # Words with highest probability will be set to suggestion
  suggestion = None

  # Initialie the value for maximum probability
  max_prob = 0

  # For each word and its probability in the probabilities dictionary
  for word, prob in probabilities.items():
    # if the optional start with string is set
    if start_with != None:
      # Check if the beginning of word does not match with the letters in 'start_with'
      if not word.startswith(start_with):
        # if they don't match, skip this word and move onto the next word
        continue

    # Check if this word's probability is greater than the current maximum probability
    if prob > max_prob:
      # if so, save this word for the best suggestion
      suggestion = word
      max_prob = prob
  
  return suggestion, max_prob




In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
# test your code when setting the starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")


The previous words are 'i like',
	and the suggested word is `a` with a probability of 0.2727

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.0909


## Get multiple suggestions

In [ ]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k = 1.0, start_with = None):
  model_counts = len(n_gram_counts_list)
  suggestions = []

  for i in range(model_counts - 1):
    n_gram_counts = n_gram_counts_list[i]
    n_plus1_gram_counts = n_gram_counts_list[i + 1]

    suggestion = suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k = k, start_with = start_with)

    suggestions.append(suggestion)

  return suggestions

In [ ]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts = count_n_grams(sentences, 3)
quadgram_counts = count_n_grams(sentences, 4)
qintgram_counts = count_n_grams(sentences, 5)

n_gram_counts_list = [unigram_counts, bigram_counts, trigram_counts, quadgram_counts, qintgram_counts]
previous_tokens = ["i", "like"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, unique_words, k=1.0)

print(f"The previous words are 'i like', the suggestions are:")
display(tmp_suggest3)

The previous words are 'i like', the suggestions are:


[('a', 0.2727272727272727),
 ('a', 0.2),
 ('cat', 0.1111111111111111),
 ('cat', 0.1111111111111111)]

# Suggest multipe words using n-gram of varyinh length

In [ ]:
n_gram_counts_list = []
for n in range(1, 6):
  print("computing n-gram counts with n = ", n, "....")
  n_model_counts = count_n_grams(train_data_processed, n)
  n_gram_counts_list.append(n_model_counts)

computing n-gram counts with n =  1 ....
computing n-gram counts with n =  2 ....
computing n-gram counts with n =  3 ....
computing n-gram counts with n =  4 ....
computing n-gram counts with n =  5 ....


In [ ]:
previous_tokens = ["i", "am", "to"]
tmp_suggest4 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest4)

The previous words are ['i', 'am', 'to'], the suggestions are:


[('be', 0.027662668120683388),
 ('have', 0.00013484358144552318),
 ('have', 0.0001348799568384138),
 ('i', 6.744907594765952e-05)]

In [ ]:
previous_tokens = ["hey", "how", "are"]
tmp_suggest6 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest6)

The previous words are ['hey', 'how', 'are'], the suggestions are:


[('you', 0.02342280731749017),
 ('you', 0.0035587188612099642),
 ('you', 0.00013488905375328792),
 ('i', 6.744907594765952e-05)]

In [ ]:
previous_tokens = ["hey", "how", "are"]
unigram_counts = n_gram_counts_list[0]
bigram_counts = n_gram_counts_list[1]
unique_words = vocabulary

tmp_suggestions = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(tmp_suggestions)

('you', 0.02342280731749017)


In [ ]:
previous_tokens = ["hey", "how", "are"]
unigram_counts = n_gram_counts_list[1]
bigram_counts = n_gram_counts_list[2]
unique_words = vocabulary

tmp_suggestions = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(tmp_suggestions)

('you', 0.0035587188612099642)


In [ ]:
previous_tokens = ["hey", "how", "are"]
unigram_counts = n_gram_counts_list[3]
bigram_counts = n_gram_counts_list[4]
unique_words = vocabulary

tmp_suggestions = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(tmp_suggestions)

('i', 6.744907594765952e-05)
